In [1]:
import numpy as np
import cupy as cp
import pathlib
import csv
import sys

In [2]:
sys.path.append(str(pathlib.Path('../').resolve()))

In [3]:
from lib.experiments import train_test_experiment, build_CNN, build_augmentation_pipeline
from lib.utils.enums import NonLinearity, Initialization, Dataset, OptimizerName
from lib.utils.data import load_integer_dataset
from lib.utils.misc import to_cpu

## Experiments config

In [4]:
architecture = [
    ('Conv2d', 128, 3, 1, 1),
    ('Conv2d', 256, 3, 1, 1),
    ('MaxPool2d', 2, 2),
    ('Conv2d', 256, 3, 1, 1),
    ('Conv2d', 512, 3, 1, 1),
    ('MaxPool2d', 2, 2),
    ('Conv2d', 512, 3, 1, 1),
    ('MaxPool2d', 2, 2),
    ('Conv2d', 512, 3, 1, 1),
    ('MaxPool2d', 2, 2),
]
num_fc_layers = 2
num_fc_hidden = [1024]

In [5]:
dataset_config = dict(
    dataset=Dataset.CIFAR10.name,
    subsample_ratio=1,
    ohe_values=(0, 32),
    data_augmentation=False,
    normalize=True,
)

In [6]:
network_config = dict(
    architecture=architecture,
    num_fc_layers=num_fc_layers,
    num_fc_hidden=num_fc_hidden,
    non_linearity=NonLinearity.NITRO_LEAKY_RELU.name,
    init=Initialization.UNIFORM_KAIMING_LEAKY_RELU.name,
    pred_decoder_dim=None,
    fe_dropout_rate=0.0,
    fc_dropout_rate=0.0,
    seed=42,
    dtype='int32',
    device='cuda',
    local_loss='pred',
    pooling_type='max',
    bias=False,
    debug=False,
)

In [7]:
training_config = dict(
    batch_size=64,
    num_epochs=50,
    lr_inv=512,
    lr_amp_factor=640,
    fwd_decay_inv=0,
    subnet_decay_inv=0,
    optimizer=OptimizerName.INTEGER_SGD.name,
)

In [8]:
config = dataset_config | network_config | training_config

In [9]:
def run_experiment(exp_config):
    print(f"{'-' * 64}")
    print(f"Running experiment with learning_dim = {exp_config['pred_decoder_dim']}")
    # Setup reproducibility
    np.random.seed(exp_config["seed"])
    cp.random.seed(exp_config["seed"])
    
    # Load the dataset
    data = load_integer_dataset(exp_config, ohe_values=exp_config["ohe_values"], val_dim=0, show_images=False, show_log=False, data_path='../.data')
    X_tr, _, X_test, y_tr, _, y_test = data

    # Setup data augmentation
    augmentation_pipeline = build_augmentation_pipeline(exp_config)

    # Build and train the model
    m = build_CNN(exp_config, X_tr=X_tr, y_tr=y_tr)    
    best_train_acc, best_test_acc = train_test_experiment(
        m, exp_config, X_tr=X_tr, X_te=X_test, y_tr=y_tr, y_te=y_test,
        augmentation_pipeline=augmentation_pipeline,
        show_local_accuracies=False, show_progress_bar=False
    )

    # Append the results to a CSV file
    with open('res/learning_dim.csv', mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([exp_config["pred_decoder_dim"], best_train_acc, best_test_acc])

In [10]:
learning_dims = [512, 1024, 2048, 4096, 8192]

In [11]:
for learning_dim in learning_dims:
    config["pred_decoder_dim"] = learning_dim 
    run_experiment(config)

----------------------------------------------------------------
Running experiment with learning_dim = 512


Files already downloaded and verified


Files already downloaded and verified


Epoch   1/50  - Train accuracy: 10.059% - Test accuracy: 09.986%


Epoch   2/50  - Train accuracy: 10.141% - Test accuracy: 10.006%


...

Epoch  46/50  - Train accuracy: 99.604% - Test accuracy: 81.941%


Epoch  47/50  - Train accuracy: 99.654% - Test accuracy: 81.861%


Epoch  48/50  - Train accuracy: 99.698% - Test accuracy: 81.781%


Epoch  49/50  - Train accuracy: 99.708% - Test accuracy: 82.101%


Epoch  50/50  - Train accuracy: 99.766% - Test accuracy: 81.380%
